# Use trulens packages from stage

In [ ]:
# Imports.
import shutil
import sys

from snowflake.snowpark.context import get_active_session

# Parameters.
TRULENS_PACKAGES = [
    "trulens-connectors-snowflake",
    "trulens-core",
    "trulens-feedback",
    "trulens-otel-semconv",
    "trulens-providers-cortex",
]
STAGE_NAME = "TRULENS_PACKAGES_STAGE"


# Helper functions.
def clear_existing_trulens_modules() -> None:
    for module_name in list(sys.modules.keys()):
        if module_name.startswith("trulens"):
            del sys.modules[module_name]


def use_package(
    trulens_package: str,
    zip_directory: str = "/tmp/zip",
    expanded_directory: str = "/tmp/expanded",
) -> None:
    snowpark_session = get_active_session()
    get_status = snowpark_session.file.get(
        f"@{STAGE_NAME}/{trulens_package}.zip", zip_directory
    )
    if len(get_status) != 1:
        raise ValueError(f"Failed to download `{trulens_package}.zip`")
    shutil.unpack_archive(
        f"{zip_directory}/{trulens_package}.zip", expanded_directory
    )
    sys.path.insert(0, expanded_directory)


# Use trulens packages from stage.
for trulens_package in TRULENS_PACKAGES:
    use_package(trulens_package)

# Test importing trulens packages

In [ ]:
import trulens.connectors.snowflake

print(trulens.connectors.snowflake.__version__)

In [ ]:
import trulens.core

print(trulens.core.__version__)

In [ ]:
import trulens.feedback

print(trulens.feedback.__version__)

In [ ]:
import trulens.providers.cortex

print(trulens.providers.cortex.__version__)

In [ ]:
import trulens.otel.semconv

print(trulens.otel.semconv.__version__)

# Run a simple app

In [ ]:
# Imports.
import time

import snowflake.snowpark as snowpark
from trulens.apps.basic import TruBasicApp
from trulens.connectors.snowflake import SnowflakeConnector
from trulens.core import TruSession
from trulens.core.feedback import Feedback
from trulens.core.feedback import SnowflakeFeedback
from trulens.providers.cortex import Cortex

snowpark_session = get_active_session()


# Helper functions.
def wait_till_feedbacks_done(
    snowpark_session: snowpark.Session,
    num_expected_feedbacks: int,
    timeout_in_seconds: int = 120,
) -> None:
    start_time = time.time()
    while time.time() - start_time < timeout_in_seconds:
        res = snowpark_session.sql(
            "SELECT STATUS FROM TRULENS_FEEDBACKS"
        ).collect()
        if len(res) == num_expected_feedbacks and all([
            curr.STATUS == "done" for curr in res
        ]):
            return
        time.sleep(1)
    raise ValueError("Feedback evaluation didn't complete in time!")


# Set up TruLens.
connector = SnowflakeConnector(
    snowpark_session=snowpark_session,
    init_server_side=False,
    database_check_revision=False,
)
tru_session = TruSession(connector=connector)

# Set up feedback functions.
relevance = Cortex(snowpark_session, retry_timeout=60).relevance
f_regular = Feedback(relevance).on_input_output()
f_snowflake = SnowflakeFeedback(relevance).on_input_output()
feedbacks = [
    f_regular,
    f_snowflake,
]

# Set up app.
tru_app = TruBasicApp(
    text_to_text=lambda _: "Tokyo is the capital of Japan.",
    feedbacks=feedbacks,
)

# Invoke app.
with tru_app:
    tru_app.main_call("What is the capital of Japan?")

# Wait.
wait_till_feedbacks_done(snowpark_session, len(feedbacks))

# Output result.
records_and_feedback = tru_session.get_records_and_feedback()
print(records_and_feedback[1])
records_and_feedback[0].T

# Finish with an obvious statement

In [ ]:
print("Kojikun is the world's cutest baby!")